In [2]:
from sqlalchemy import create_engine, select, update
from sqlalchemy.orm import Session, joinedload
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import api.entities as db
from itemchm.entities_repr import User, Book

# Create engine
ENGINE = create_engine("sqlite:///../../bookshelf.db", echo=True)

In [3]:
def sentiment_analysis(text : str):
    # Create a SentimentIntensityAnalyzer object
    sia = SentimentIntensityAnalyzer()

    # Perform sentiment analysis
    sentiment_scores = sia.polarity_scores(text)
    
    if sentiment_scores['compound'] < -0.1:
        return 1
    elif sentiment_scores['compound'] < 0.1:
        return 4
    else:
        return 6
        
def overall_rate(user_book : db.UserBook) -> float:
    RATING_WEIGHT = 4
    COMMENT_WEIGHT = 3
    SHARED_WEIGHT = 2
    READ_RATIO_WEIGHT = 1

    if user_book.rating == None:
        rating_value = 4
    else:
        rating_value = user_book.rating + 1
    
    if user_book.comment == None:
        comment_value = 4
    else:
        comment_value = sentiment_analysis(user_book.comment)
    
    read_ration_value = user_book.readRatio * 5 + 1

    if user_book.shared <= 0:
        shared_value = 1
    elif user_book.shared < 3:
        shared_value = 2
    elif user_book.shared < 6:
        shared_value = 3
    elif user_book.shared < 8:
        shared_value = 4
    elif user_book.shared < 12:
        shared_value = 5
    else:
        shared_value = 6

    return (RATING_WEIGHT * rating_value + COMMENT_WEIGHT * comment_value + SHARED_WEIGHT * shared_value + READ_RATIO_WEIGHT * read_ration_value) / (RATING_WEIGHT + COMMENT_WEIGHT + SHARED_WEIGHT + READ_RATIO_WEIGHT)

In [4]:
with Session(ENGINE) as session:
    db_users = session.query(db.User).all()
    db_books = session.query(db.Book).options(joinedload(db.Book.genres)).all()
    db_user_books = session.query(db.UserBook).all()

2024-08-27 12:30:22,612 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-27 12:30:22,615 INFO sqlalchemy.engine.Engine SELECT "Users".id AS "Users_id", "Users".name AS "Users_name", "Users".email AS "Users_email", "Users".features AS "Users_features" 
FROM "Users"
2024-08-27 12:30:22,616 INFO sqlalchemy.engine.Engine [generated in 0.00110s] ()
2024-08-27 12:30:22,622 INFO sqlalchemy.engine.Engine SELECT "Books".id AS "Books_id", "Books".name AS "Books_name", "Books".author AS "Books_author", "Books".language AS "Books_language", "Books".year AS "Books_year", "BookGenre_1"."booksId" AS "BookGenre_1_booksId", "BookGenre_1"."genresName" AS "BookGenre_1_genresName" 
FROM "Books" LEFT OUTER JOIN "BookGenre" AS "BookGenre_1" ON "Books".id = "BookGenre_1"."booksId"
2024-08-27 12:30:22,623 INFO sqlalchemy.engine.Engine [generated in 0.00081s] ()
2024-08-27 12:30:22,702 INFO sqlalchemy.engine.Engine SELECT "UserBooks".id AS "UserBooks_id", "UserBooks"."readRatio" AS "UserBooks_readRatio",

In [5]:
users_dict : dict[int, User] = {}

for db_user in db_users:
    user = User(db_user.id, db_user.name, {})
    users_dict[db_user.id] = user

books : list[Book] = []

for db_book in db_books:
    genres = [book_genre.genresName for book_genre in db_book.genres]
    book = Book(db_book.id, db_book.name, db_book.author, db_book.year, db_book.language, genres)
    books.append(book)

In [8]:
for user_book in db_user_books:
    rating = overall_rate(user_book)
    user = users_dict[user_book.userId]
    user.ratings[user_book.bookId] = rating

In [10]:
# Required Input
users = list(users_dict.values())
books

{1519: 5.265,
 26297: 5.4,
 17816: 5.05,
 26487: 5.05,
 60479: 5.21,
 5720: 5.385,
 38145: 5.365,
 932: 5.7299999999999995,
 12242: 5.2700000000000005,
 28054: 5.58,
 5: 2.12,
 35451: 1.3599999999999999,
 1952: 5.3549999999999995,
 967: 4.84,
 1661: 5.1049999999999995,
 14033: 2.1350000000000002,
 42078: 5.985,
 61851: 5.3950000000000005,
 73955: 5.1,
 19: 5.45,
 972: 5.13,
 28555: 5.0200000000000005,
 1549: 2.125,
 284: 5.04,
 10148: 5.585,
 74111: 1.4,
 3691: 5.79,
 10623: 5.16,
 16: 5.13,
 23962: 2.1350000000000002,
 16357: 5.41,
 60852: 1.405,
 50742: 1.225,
 42290: 1.33,
 53436: 4.63,
 31552: 5.535,
 2946: 5.41,
 41537: 2.1350000000000002,
 27107: 1.825,
 3825: 6.0,
 4093: 5.029999999999999,
 3160: 1.285,
 52263: 5.255,
 61130: 1.225,
 257: 2.07,
 31509: 2.12,
 5200: 4.955,
 59844: 4.779999999999999,
 12: 4.505,
 46800: 1.645,
 994: 5.255,
 910: 5.16,
 15489: 4.835,
 9380: 1.875,
 73952: 5.015,
 42324: 4.425,
 6400: 1.16,
 1237: 5.455,
 41360: 5.01,
 23979: 5.925,
 74070: 1.365,
 

In [ ]:
from itemchm.hybrid_recommender import HybridRecommender


system = HybridRecommender(books, users)

selected_user = User(1241241241, "BritoAlv", { 4 : 5})

result = system.recommend(selected_user, 5)

for book in result:
    print(book)

print("")
print(selected_user)